## Utilizing NHANES dataset 2021-2023 for data analysis
### I sought to predict hemoglobin A1c levels (an indicator of having diabetes) utilizing diet and demographic information
### The dataset can be found at [diet](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DR1IFF_L.htm) [demographics](https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Demographics&Cycle=2021-2023) and [hemoglobin](https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Laboratory&Cycle=2021-2023)


### The following libraries below will be used for data analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.ensemble import RandomForestClassifier

import plotly.graph_objects as go



### The filepaths for the diet, demographics, and hemoglobin A1C are shown below

In [ ]:
diet_path = '/Users/velarieansu/Documents/datascience/DR1IFF_L.xpt'
demo_path = '/Users/velarieansu/Documents/datascience/DEMO_L.xpt'
a1c_path = '/Users/velarieansu/Documents/datascience/GHB_L.xpt'

### Datasets were downloaded as SAS xport files
#### Additionally, the head function was used to preview the dataframe for a quick inspection

In [ ]:
df_demo = pd.read_sas(demo_path, format='xport', encoding='utf-8')
print(df_demo.head())

In [ ]:
df_a1c = pd.read_sas(a1c_path, format='xport', encoding='utf-8')
print(df_a1c.head())

In [ ]:
df_diet = pd.read_sas(diet_path, format='xport', encoding='utf-8')
print(df_diet.head())


#### Info was used for each dataset to provide a summary of the dataframe to help understand the structure

In [ ]:
df_demo.info()
df_diet.info()
df_a1c.info()


### The describe() function was used to generate descriptive statistics of specific numerical variables

In [ ]:
df_a1c[["LBXGH"]].describe()


In [ ]:
df_demo[["RIDAGEYR"]].describe()

In [ ]:
df_diet[['DR1IKCAL','DR1IPROT','DR1ICARB', 'DR1ITFAT' ]].describe()

### value_counts() shows the summary of categorical data of gender below

In [ ]:
print(df_demo['RIAGENDR'].value_counts())

### Datasets have many columns that are not relevant to this study thus selecting columns of interest 

In [ ]:
# columns to keep 
demo = df_demo[['SEQN','RIAGENDR','RIDAGEYR']]

In [ ]:
diet = df_diet[['SEQN','DR1IKCAL','DR1IPROT','DR1ICARB', 'DR1ITFAT']]

In [ ]:
a1c= df_a1c [['SEQN', 'LBXGH']]

In [ ]:
#### The value_counts function was used to count all the different gender of the participants.
#### coding of 2 = Female and coding of 1 = Male
#### See Link for details

In [ ]:
demo.RIAGENDR.value_counts()

#### The variable name is replaced and the coded values of 1 and 2 are assigned to Male and Female respective;y

In [ ]:
### demo["Gender"] = demo.RIAGENDR.replace({1: "Male", 2: "Female"})

#### Checking to make sure assignemnts were done correctly

#### Checking for null and na for all dataframes

In [ ]:
demo.isna().sum()


In [ ]:
diet.isna().sum()


In [ ]:
a1c.isna().sum()

#### From above, it can be seen that there were no missing values for demographics, however, dietary data and hemoglobin A1C variables had 329 and 484 missing variables respectively

#### A1C levels will be converted into categorical variables based on clinical recommendations

#### Writing a function to categorize A1C levels

In [ ]:
# Categorize A1c

df_cat = a1c
def a1c_levels(value):
    if value < 5.7:
        return 'Normal'
    elif 5.7 <= value < 6.5:
        return 'Prediabetes'
    else:
        return 'Diabetes'

        


### Create a new colum a1c_category to show the various levels of a1c categories

In [ ]:
a1c['a1c_category'] = a1c['LBXGH'].apply(a1c_levels)

# View the result
print(a1c.head())

In [ ]:
# Drop rows with missing target
a1c = a1c.dropna(subset=['LBXGH'])

### Drop rows with missing features

In [ ]:
demo = demo.dropna(subset=['RIAGENDR','RIDAGEYR'])


In [ ]:
diet = diet.dropna(subset=['DR1ICARB','DR1IPROT','DR1ITFAT','DR1IKCAL'])

In [ ]:
print(data[features].shape)
print(data['a1c_label'].shape)


In [ ]:
# Rechecking missing variables for target 
a1c.isna().sum()

In [ ]:
# Rechecking missing variables for features
diet.isna().sum()

In [ ]:
demo.isna().sum()

In [ ]:
print(demo.shape)
print(diet.shape)
print(a1c.shape)

### Encode categorical variables so that ML algorithms can be processed
#### LabelEncoder() creates an encoder object that converts categorical labels into numeric 
#### fit_transform() will finds all unique values and will assign each one a number, and replaces the text with that number.

In [ ]:

le = LabelEncoder()
demo['gender_encoded'] = le.fit_transform(demo['Gender'])  # male =1 female =0

# Encode target variable
a1c['a1c_encoded'] = le.fit_transform(a1c['a1c_category'])  # Normal = 1, Prediabetes = 2, Diabetes = 3




In [ ]:
a1c.head()

In [ ]:
demo.head()

### Merge diet and demographic data

In [ ]:
merged_temp = pd.merge(diet, demo, on='SEQN', how='inner')
merged_all = pd.merge(merged_temp, a1c, on='SEQN', how='inner')

# Check result
print(merged_all.shape)
print(merged_all.head())

## Showing some visualizations

### A1c distribution

In [ ]:
label_map = {
    0: 'Normal',
    1: 'Prediabetes',
    2: 'Diabetes'
}
merged_all['a1c_category_label'] = merged_all['a1c_encoded'].map(label_map)

sns.countplot(data=merged_all, x='a1c_category_label', palette='Set2')
plt.title("Distribution of A1C Categories")
plt.xlabel("A1C Category")
plt.ylabel("Count")
plt.show()


### From the histogram above, it can be observed that the participants classified as Normal (No diabetes) have the lowest count, as compared to Prediabetes(1) and diabetes (2) groups.

### Boxplots below: Macronutrients by A1C Category

In [ ]:
sns.boxplot(data=merged_all, x='a1c_category_label', y='DR1ICARB', palette='pastel')
plt.title("Carbohydrate Intake Across A1C Categories")
plt.ylabel("Carbohydrates (grams)")
plt.xlabel("A1C Category")
plt.show()


In [ ]:
sns.boxplot(data=merged_all, x='a1c_category_label', y='DR1IPROT', palette='pastel')
plt.title("Proteins Intake Across A1C Categories")
plt.ylabel("Proteins (grams)")
plt.xlabel("A1C Category")
plt.show()


In [ ]:
sns.boxplot(data=merged_all, x='a1c_category_label', y='DR1ITFAT', palette='pastel')
plt.title("Fat Intake Across A1C Categories")
plt.ylabel("Fat (grams)")
plt.xlabel("A1C Category")
plt.show()


### Heatmaps for correlations

In [ ]:
features = ['RIDAGEYR', 'gender_encoded', 'DR1ICARB', 'DR1IPROT', 'DR1ITFAT', 'DR1IKCAL']
corr = merged_all[features].corr()

sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Correlation Matrix of Dietary Features")
plt.show()


### Age vs Total Calories Colored by A1C

### Making the selection for the target and features 

In [ ]:
features = ['RIDAGEYR', 'gender_encoded', 'DR1ICARB', 'DR1IPROT', 'DR1ITFAT', 'DR1IKCAL']
X = merged_all[features]
y = merged_all['a1c_encoded']


In [ ]:
print(X.shape)
print(y.shape)

### Split data
#### Data splitting in machine learning helps in the creation of separate datasets which are critical when data is trained, validated, and tested. This is beneficial for the evaluation of model performance as well as help in preventing overfitting.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Scaling features also known as Z-score normalization
#### In ML, this method is important in data preprocessing.
#### Here, each feature is rescaled such that it has a standard deviation of 1 and a mean of 0


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### Model Training
#### In ML, models are trained so that they are able to learn the patterns/relationships in the data and transfer that by making accurate predictions in new data.

In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)


### Model Evaluation
#### We determine how well the trained model above performs its task on the unseen data.
#### Thus, evaluating the classification model that predicts whether a person is Normal, Prediabetes, or has Diabetes based on the test features.

In [ ]:
y_pred = model.predict(X_test_scaled)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['Normal', 'Prediabetes', 'Diabetes']))


 ### The results above shows the confusion matrix. 
#### Precision shows how many were actually in the class
#### Recall shows how many were correctly identified
#### F1-score combines precision and recall into one metric -- balanced metric
#### Support is the number of true instances
#### Based on the output, out of 1722 for normal, only 238 were predicted-- most of them were predicted as prediabetes or diabetes
#### Overall, prediabetes were classified as best with f1-score of 0.76, normal is worse recall 14%
#### The overall accuracy shows 63% for correct predictions

#### The model is biased toward the dominant class prediabetes most likely due to the imbalance
Perhaps different models need to be experimented 

### Feature Importance

### As the name implies, helps to understand the features with greatest influence on a models prediction

In [ ]:
importances = pd.Series(model.feature_importances_, index=features)
importances.sort_values().plot(kind='barh')
plt.title('Feature Importance')
plt.show()



### Utilizing class weights
#### Method is chosen because it does not alter the data distribution making it 
   ####  important for clinical interpretability.

In [ ]:



# Use a classifier with class weights
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=['Normal', 'Prediabetes', 'Diabetes']))
print(confusion_matrix(y_test, y_pred))


#### Results still shows some improvement over previous model
#### We can see that the old recall of 0.14 is now 0.23 an the old f1-score 0.20 is now 0.24
#### Accuracy decreased slightly but overall this model looks fair mostly for the detection of those classified as Normal and Diabetes

### Visualizing the Confusion Matrix

In [ ]:



# Display confusion matrix with class names and colors
ConfusionMatrixDisplay.from_predictions(
    y_test,
    y_pred,
    display_labels=['Normal', 'Prediabetes', 'Diabetes'],
    cmap='Blues',
    values_format='d'  # Format as integers
)

plt.title("Confusion Matrix - Random Forest with Balanced Class Weights")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.tight_layout()
plt.show()


In [ ]:
# interactive with plotly
!pip install plotly


In [ ]:


# Get confusion matrix
labels = ['Normal', 'Prediabetes', 'Diabetes']
cm = confusion_matrix(y_test, y_pred)

# Create interactive heatmap
fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=labels,
    y=labels,
    hoverongaps=False,
    colorscale='Blues',
    showscale=True,
    text=cm,
    texttemplate="%{text}"
))

fig.update_layout(
    title="Interactive Confusion Matrix - Random Forest",
    xaxis_title="Predicted Label",
    yaxis_title="True Label",
    yaxis=dict(autorange='reversed')  # Make top row 'Normal'
)

fig.show()
